In [1]:
import pandas as pd
import numpy as np
import os
from functools import reduce

### ESL Server Server Name: PMPLWAAA9254\SQLP01

In [2]:
import win32com.client as win32
from datetime import datetime
import os

In [3]:
df1 = pd.read_csv('DataCheckFields.csv', engine ='python')
dfback = df1  #Unflitered version of df1#
df1.columns

Index(['UwrMDSLScenarioId', 'EffectiveDate', 'AccountID', 'AccountName',
       'UnderwriterName', 'UwrAssistantName', 'QuoteStatusDescription',
       'IsRenewal', 'RenewalTier', 'ReasonStatusDescription',
       'MostLikelyProposal', 'Scenario', 'Description', 'Status', 'CensusEE',
       'CensusES', 'CensusEC', 'CensusEF', 'SpIsGapLessRenewal',
       'IsNumberOfSpouseChildrenOverride', 'SpContractType', 'OptionSequence',
       'SpSpecialCompensation', 'SpCommission', 'SpPremiumRefund',
       'CurrentAgeFactor', 'CurrentDepBaseRate', 'SpIsPremiumRefund',
       'SpRefundSharePercent', 'SpLossRatio', 'SpCapPercent',
       'SpIsNetCommission', 'Census', 'StrategicPartner',
       'ClaimAdministrator1', 'ClaimAdministrator1Fee',
       'ClaimAdministrator1Lives', 'ClaimAdministrator2',
       'ClaimAdministrator2Fee', 'ClaimAdministrator2Lives',
       'ClaimAdministrator3', 'ClaimAdministrator3Fee',
       'ClaimAdministrator3Lives', 'ClaimAdministrator4',
       'ClaimAdministrato

In [4]:
df1=df1[((df1['QuoteStatusDescription'] == 'Quoted Dead') | (df1['QuoteStatusDescription'] == 'Declined') |(df1['QuoteStatusDescription'] == 'Quoted')|(df1['QuoteStatusDescription'] == 'Sold') | (df1['QuoteStatusDescription'] == 'In Underwriting'))] 
#df1

convert_dtype = {'IsRenewal': bool,
                'SpIsGapLessRenewal': bool,
                'IsNumberOfSpouseChildrenOverride': bool,
                'SpIsPremiumRefund': bool,
                'SpIsNetCommission': bool,
               }  
df1 = df1.astype(convert_dtype)  

### ESL Data Check 1

-  "Strategic Partner", is blank

In [5]:
df1_con = df1[(df1['StrategicPartner'].isnull() == True) & (df1['OptionSequence']==1)& (df1['OptionSequence']==1)][['AccountID','AccountName','Description','Scenario','OptionSequence','Status','EffectiveDate','UnderwriterName', 'UwrAssistantName','StrategicPartner','QuoteStatusDescription']].drop_duplicates()
df1_con.to_csv('StrategicPartner1.csv')
df1_con['FailReason'] = 'Group does not have a strategic Partner'
with open('StrategicPartner1.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #1: Groups Missing a Strategic Partner' + '\n' +'\n' + content)
df1_con


,AccountID,AccountName,Description,Scenario,OptionSequence,Status,EffectiveDate,UnderwriterName,UwrAssistantName,StrategicPartner,QuoteStatusDescription,FailReason
192,34539,"Burton Plumbing Services, LLC",35k 24/12,1,1,OPEN,01/01/2022,"Smith, Anna","Swiney, Toral",NaN,Declined,Group does not have a strategic Partner
472,34609,Cable Dahmer Auto Group,100k 12/12,1,1,OPEN,01/01/2022,"Smith, Anna","Swiney, Toral",NaN,Declined,Group does not have a strategic Partner
940,22488,"U.A. Local No. 290 Plumber, Steamfitter & Shi...",Options,2,1,LOCK,01/01/2022,"Brandsness, Eric","Flores-Ortiz, Stephanie",NaN,Quoted,Group does not have a strategic Partner
1694,36158,Bracalente Manufacturing Group,"55K, 24/12",1,1,OPEN,01/01/2022,"Nickolas, David","Ramadas, Harshitha",NaN,Declined,Group does not have a strategic Partner
1798,21003,"Mill Creek Residential Trust Resources, LLC (MCR)",Firm Quote,2,1,LOCK,01/01/2022,"Nickolas, David","Bates, Erick",NaN,Quoted,Group does not have a strategic Partner
...,...,...,...,...,...,...,...,...,...,...,...,...
51090,34597,"Morris and Dickson Co, LLC","200K, 15/12",1,1,LOCK,01/01/2022,"Nanson, Jonathan","Brackett, Mary",NaN,In Underwriting,Group does not have a strategic Partner
51169,7470,Aspen Valley Hospital,Current Year,2,1,LOCK,01/01/2022,"Brandsness, Eric","Herda, Luke",NaN,In Underwriting,Group does not have a strategic Partner
52711,28720,"Syska Hennessy Group, Inc.",199k 24/12,1,1,LOCK,01/01/2022,"Nickolas, David","Bates, Erick",NaN,In Underwriting,Group does not have a strategic Partner
53085,27551,Prosser Memorial Hospital FDN,Adding Commissions,2,1,LOCK,01/01/2022,"Manning, Alexandra","Flores-Ortiz, Stephanie",NaN,Quoted,Group does not have a strategic Partner


### ESL Data Check 2

- Rate to Rate Report:("Administrator 1", OR "Administrator 2", OR "Administrator 3", OR "Administrator 4", OR "Administrator 5") = UMR
- Renewal Reprt "Current Cost containment" <> 0.95

In [6]:
# -- Filter for UMR --#

df2_con = df1[((df1['ClaimAdministrator1'] == 'UMR') | (df1['ClaimAdministrator2'] == 'UMR') | 
               (df1['ClaimAdministrator3'] == 'UMR') | (df1['ClaimAdministrator4'] == 'UMR') | 
               (df1['ClaimAdministrator5'] == 'UMR')) & (df1['SpCostContainmentAdj'] != 0.95)][['AccountID',
                  'AccountName','Description','QuoteStatusDescription','Scenario',
                  'OptionSequence','Status','EffectiveDate','SpCostContainmentAdj','UnderwriterName','UwrAssistantName',
                  'ClaimAdministrator1','ClaimAdministrator2','ClaimAdministrator3',
                  'ClaimAdministrator4','ClaimAdministrator5']].drop_duplicates()
df2_con['FailReason'] = 'UMR is selected as the administration but Current Cost Containment set to .95'  
# --Write to File --#   
df2_con.to_csv('CurrentCostContainment2.csv')
with open('CurrentCostContainment2.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #2: Groups that have UMR as the administrator and do not have Current Cost Containment set to .95' + '\n' +'\n' + content)
df2_con

,AccountID,AccountName,Description,QuoteStatusDescription,Scenario,OptionSequence,Status,EffectiveDate,SpCostContainmentAdj,UnderwriterName,UwrAssistantName,ClaimAdministrator1,ClaimAdministrator2,ClaimAdministrator3,ClaimAdministrator4,ClaimAdministrator5,FailReason
126,35067,City of Henderson,525k 24/12,In Underwriting,1,3,LOCK,01/01/2022,0.90,"Tollis, Elizabeth","Bates, Erick","HealthSCOPE Benefits, Inc.",UMR,Express Scripts (RX),NaN,NaN,UMR is selected as the administration but Curr...
242,30627,"CareStar, Inc.",copy of scenario 2 updated base rates,Quoted,3,3,LOCK,01/01/2022,0.94,"Eichler, Mark","Lee, Pa",UMR,NaN,NaN,NaN,NaN,UMR is selected as the administration but Curr...
280,16100,"Truck Centers, Inc.","150k, 24/12",In Underwriting,1,3,LOCK,02/01/2022,1.00,"Smith, Anna","Flores-Ortiz, Stephanie",UMR,NaN,NaN,NaN,NaN,UMR is selected as the administration but Curr...
305,16645,"Community Coffee Company, LLC",Current Year,In Underwriting,2,1,LOCK,01/01/2022,0.90,"Onufer, Max","Garrod, Quinn",UMR,OptumRx Claims,NaN,NaN,NaN,UMR is selected as the administration but Curr...
317,6596,WB Holdings Inc.,Current Year,In Underwriting,2,1,LOCK,01/01/2022,0.90,"Aiello, Rhonda","Garrod, Quinn",UMR,OptumRx Claims,NaN,NaN,NaN,UMR is selected as the administration but Curr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52859,19435,Fellowship of Catholic University Students,UMR option,Sold,4,3,OPEN,07/01/2022,0.90,"Tollis, Elizabeth","Bates, Erick",UMR,NaN,NaN,NaN,NaN,UMR is selected as the administration but Curr...
52920,21192,Horizon Retail Construction,2022 Renewal,In Underwriting,2,3,LOCK,01/01/2022,0.94,"Smith, Anna","Lee, Pa",UMR,Caremark (RX),NaN,NaN,NaN,UMR is selected as the administration but Curr...
53062,16645,"Community Coffee Company, LLC",Copy SCN 2 - due to base rate update,Quoted,4,3,LOCK,01/01/2022,0.90,"Onufer, Max","Garrod, Quinn",UMR,OptumRx Claims,NaN,NaN,NaN,UMR is selected as the administration but Curr...
53077,6555,Furst-McNess Company,Changing BOR and to 1/1 eff date,In Underwriting,4,2,OPEN,01/01/2022,0.90,"Smith, Anna",NaN,UMR,NaN,NaN,NaN,NaN,UMR is selected as the administration but Curr...


### ESL Data Check # 3

- Rate to Rate Report:("PPO Network 1", OR "PPO Network 2", OR "PPO Network 3", OR "PPO Network 4", OR "PPO Network 5") starts with "2021"

In [7]:
# -- Check that UMR Fields have been entered correctly --#

# -- Pull in Columns for Test -- #
PPO = df1[['AccountID','AccountName','Description','QuoteStatusDescription','EffectiveDate','PPONetworkDescription',
           'PPOSequence','Scenario','OptionSequence','Status','UnderwriterName', 'UwrAssistantName', ]]

# -- Pivot by PPOSequence -- #
PPOPivot=PPO.pivot_table(index=['AccountID','AccountName','Description','QuoteStatusDescription','Scenario','Status','OptionSequence',
                                'EffectiveDate','UnderwriterName', 'UwrAssistantName'], columns='PPOSequence', 
                                 values='PPONetworkDescription', aggfunc='first')

# -- Write Pivoted table to file and then read back CSV -- Removes Pivot Index -- #
PPOPivot.to_csv('PPOCHECK.csv')
PPOPivot=pd.read_csv('PPOCHECK.csv')


# -- Filter to show groups with a PPO that starts with 2021 -- #
df3_con = PPOPivot[(PPOPivot['OptionSequence']==1) & ((PPOPivot['1.0'].str.startswith('2021') == True) | 
                    (PPOPivot['2.0'].astype(str).str.startswith('2021') == True) | 
                    (PPOPivot['3.0'].astype(str).str.startswith('2021') == True) | 
                    (PPOPivot['4.0'].astype(str).str.startswith('2021') == True) | 
                    (PPOPivot['5.0'].astype(str).str.startswith('2021') == True) | 
                    (PPOPivot['6.0'].astype(str).str.startswith('2021') == True) | 
                    (PPOPivot['7.0'].astype(str).str.startswith('2021') == True))]
df3_con=df3_con[['AccountName','Scenario','OptionSequence','UnderwriterName','Description','QuoteStatusDescription','Status','UwrAssistantName',
                 'EffectiveDate','1.0','2.0','3.0','4.0','5.0','7.0']].drop_duplicates()
df3_con['FailReason'] = 'PPO Network should not start with 2021'
# -- Write result to file -- #
df3_con.to_csv('PPODate3.csv')
with open('PPODate3.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #3: PPO Network Starts with 2021' + '\n' +'\n' + content)
df3_con

C:\Users\i706522\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,AccountName,Scenario,OptionSequence,UnderwriterName,Description,QuoteStatusDescription,Status,UwrAssistantName,EffectiveDate,1.0,2.0,3.0,4.0,5.0,7.0,FailReason
255,Brown Brothers Harriman & Co.,3,1,"Nickolas, David",Copy of scenario 2 to fix age/sex,Quoted,OPEN,"Her, Susan",01/01/2022,NaN,2021-01 United HealthCare,NaN,NaN,NaN,NaN,PPO Network should not start with 2021
752,M.A. Mortenson Company,1,1,"Eichler, Mark","500000, 24/12",In Underwriting,LOCK,"Her, Susan",01/01/2022,NaN,NaN,NaN,2021-01 United HealthCare,NaN,NaN,PPO Network should not start with 2021
756,M.A. Mortenson Company,2,1,"Eichler, Mark",test,In Underwriting,OPEN,"Her, Susan",01/01/2022,NaN,NaN,NaN,2021-01 United HealthCare,NaN,Unevaluated PPO,PPO Network should not start with 2021
760,M.A. Mortenson Company,3,1,"Eichler, Mark",Thru Sept To firm,In Underwriting,LOCK,"Her, Susan",01/01/2022,NaN,NaN,2021-01 United HealthCare,NaN,Unevaluated PPO,NaN,PPO Network should not start with 2021
1492,Societe General,1,1,"Kusiak, Andrew","275000, 24/12",Declined,OPEN,"Mitchell, Zachary",01/01/2022,NaN,NaN,2021-01 United HealthCare,NaN,NaN,NaN,PPO Network should not start with 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39092,"Mirion Technologies, Inc",1,1,"Manning, Alexandra","150k, 12/18",In Underwriting,LOCK,"Curry, Morgan",01/01/2022,NaN,NaN,2021-01 BlueCross BlueShield (State),NaN,NaN,NaN,PPO Network should not start with 2021
39095,"Mirion Technologies, Inc",3,1,"Manning, Alexandra",Relook at Aggregate,Quoted,LOCK,"Curry, Morgan",01/01/2022,NaN,NaN,2021-01 BlueCross BlueShield (State),NaN,NaN,NaN,PPO Network should not start with 2021
39447,Brigade Energy Services LLC,1,1,"Kusiak, Andrew",150k 24/12,In Underwriting,LOCK,"Swiney, Toral",01/01/2022,NaN,NaN,2021-01 United Healthcare Choice Plus,NaN,NaN,NaN,PPO Network should not start with 2021
42760,"Full Sail, LLC",1,1,"Gross, Jennifer","250000, 24/12",In Underwriting,LOCK,"Her, Susan",04/01/2022,NaN,NaN,NaN,NaN,2021-01 United Healthcare Choice Plus,NaN,PPO Network should not start with 2021


# ESL Data Check  #4

-  "Current Dep Base Rate" < 1.0

In [8]:
# -- Pull in Columns for Test -- #

df4_con = df1[(df1['EffectiveDate']>'01/31/2022')& (df1['CurrentDepBaseRate']< 1.0)][['AccountID','AccountName',
             'Description','QuoteStatusDescription','Scenario','OptionSequence','Status','UnderwriterName',
             'UwrAssistantName','EffectiveDate','CurrentDepBaseRate']].drop_duplicates()
df4_con['FailReason'] = 'Current Dependent Base Rate is less than 1.0'
# -- Write Result to File -- #
df4_con.to_csv('CurrentDepBaseRate4.csv')
with open('CurrentDepBaseRate4.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #4: Current Dependent Base Rate less than 1.0' + '\n' +'\n' + content)

df4_con

,AccountID,AccountName,Description,QuoteStatusDescription,Scenario,OptionSequence,Status,UnderwriterName,UwrAssistantName,EffectiveDate,CurrentDepBaseRate,FailReason
3472,33251,"I.S.E.C., Inc.",#4 - all options @120%,Quoted,9,3,LOCK,"Nickolas, David","Ramadas, Harshitha",07/01/2022,0.0,Current Dependent Base Rate is less than 1.0
3591,18517,"Sunrise Community, Inc.","150000, 12/15",In Underwriting,1,2,LOCK,"Gross, Jennifer","Her, Susan",04/01/2022,0.0,Current Dependent Base Rate is less than 1.0
6560,4848,Greater Metropolitan Hotel Employers-Employees...,"400000, 24/12",Declined,1,4,OPEN,"Eichler, Mark","Bhovar, Manish",04/01/2022,0.0,Current Dependent Base Rate is less than 1.0
7886,33251,"I.S.E.C., Inc.",#4 - all options @120%,Quoted,9,6,LOCK,"Nickolas, David","Ramadas, Harshitha",07/01/2022,0.0,Current Dependent Base Rate is less than 1.0
7893,33251,"I.S.E.C., Inc.",#4 - all options @120%,Quoted,9,1,LOCK,"Nickolas, David","Ramadas, Harshitha",07/01/2022,0.0,Current Dependent Base Rate is less than 1.0
8060,36573,Carteret Board of Education,DO NOT USE 150k. 15/12,In Underwriting,2,1,OPEN,"Tollis, Elizabeth","Mullaney-Tanhviseth, Krisana",07/01/2022,0.0,Current Dependent Base Rate is less than 1.0
9527,18254,"Cardiology Consultants of Philadelphia, P.C.","115K, 24/12",Declined,1,2,OPEN,"Onufer, Max","Subramani, Santhosh",03/01/2022,0.0,Current Dependent Base Rate is less than 1.0
10722,18821,IBEW-NECA Southwestern Health & Benefit Fund,"600000, 24/12",In Underwriting,1,3,LOCK,"Nickolas, David","Bhovar, Manish",04/01/2022,0.0,Current Dependent Base Rate is less than 1.0
10823,29655,Bright Wood Corporation,"225000, 24/12",In Underwriting,1,1,OPEN,"Manning, Alexandra","Curry, Morgan",05/01/2022,0.0,Current Dependent Base Rate is less than 1.0
11119,33251,"I.S.E.C., Inc.",#4 - all options @120%,Quoted,9,5,LOCK,"Nickolas, David","Ramadas, Harshitha",07/01/2022,0.0,Current Dependent Base Rate is less than 1.0


# ESL Data Check  #5

- "Current Age Factor" < 0.50

In [9]:
# -- Pull in Columns for Test -- #

df5_con = df1[(df1['CurrentAgeFactor']<.5)& (df1['OptionSequence']==1)][['AccountID','AccountName','Description',
              'QuoteStatusDescription','Scenario','OptionSequence','Status','EffectiveDate','UnderwriterName',
              'UwrAssistantName','CurrentAgeFactor']].drop_duplicates()
df5_con['FailReason'] = 'Current Age Factor is less than .50'
# -- Write Result to File -- #
df5_con.to_csv('CurrentAgeFactor5.csv')
with open('CurrentAgeFactor5.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #5: Current Age Factor is less than .50' + '\n' +'\n' + content)
df5_con

,AccountID,AccountName,Description,QuoteStatusDescription,Scenario,OptionSequence,Status,EffectiveDate,UnderwriterName,UwrAssistantName,CurrentAgeFactor,FailReason
786,34728,Dutch Bros Coffee,"150K, 12/15",In Underwriting,1,1,LOCK,01/01/2022,"Manning, Alexandra","Evenhouse, George",0.456,Current Age Factor is less than .50
1128,19435,Fellowship of Catholic University Students,Cigna option,Quoted,6,1,OPEN,07/01/2022,"Tollis, Elizabeth","Bates, Erick",0.438,Current Age Factor is less than .50
1391,36077,Emory University,"350000, 12/18",Declined,1,1,OPEN,07/15/2022,"Nanson, Jonathan","Bhovar, Manish",0.427,Current Age Factor is less than .50
1567,31555,ScribeAmerica aka Health Channels,"100000, 12/18",In Underwriting,1,1,LOCK,01/01/2022,"Gross, Jennifer","Her, Susan",0.421,Current Age Factor is less than .50
1726,34960,Bytedance Inc.,"150000, 12/12",Declined,1,1,OPEN,01/01/2022,"Manning, Alexandra","Curry, Kenzie",0.451,Current Age Factor is less than .50
...,...,...,...,...,...,...,...,...,...,...,...,...
50842,34505,Buildertrend Solutions,"75K, 12/12 with CoConstruct Group",Declined,2,1,OPEN,01/01/2022,"Smith, Anna","Mitchell, Zachary",0.381,Current Age Factor is less than .50
51233,15146,"Pomeroy Technologies, LLC","300K, 24/12",In Underwriting,1,1,LOCK,01/01/2022,"Nanson, Jonathan","Brackett, Mary",0.416,Current Age Factor is less than .50
52600,35960,Attentive Mobile Inc.,"200000, 12/12",In Underwriting,1,1,LOCK,12/31/2022,"Nickolas, David","Bhovar, Manish",0.458,Current Age Factor is less than .50
52837,20134,Regents of the University of California,Firm Request,In Underwriting,3,1,LOCK,07/01/2022,"Garrod, Quinn","Herda, Luke",0.496,Current Age Factor is less than .50


# ESL Data Check  #6

- "Gapless Renewal" = Yes
- "Contract Type" = 12/24, 12/12, 15/12, 18/12, 24/12, Paid or other


In [10]:
# -- Pull in Columns for Test -- #
df6_con = df1[(df1['SpIsGapLessRenewal']==True) & ((df1['SpContractType'] == '1224') |
              (df1['SpContractType']=='1212') | (df1['SpContractType']=='1512') |
              (df1['SpContractType']=='1812') | (df1['SpContractType']=='2412') |
              (df1['SpContractType']=='PAID') | (df1['SpContractType']=='OTHR'))][['AccountID',
              'AccountName','Description','QuoteStatusDescription','Scenario','OptionSequence',
              'Status','EffectiveDate','UnderwriterName','UwrAssistantName','SpContractType',
              'SpIsGapLessRenewal']].drop_duplicates()
df6_con['FailReason'] = 'Gapless Renewal with an Incorrect Contract Type'
# -- Write to File -- #
df6_con.to_csv('GaplessRenewal6.csv')
with open('GaplessRenewal6.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #6: Groups with Gapless Renewal and a Contract Type of 12/24, 12/12, 15/12, 18/12, 24/12, Paid or other' + '\n' +'\n' + content)
df6_con

,AccountID,AccountName,Description,QuoteStatusDescription,Scenario,OptionSequence,Status,EffectiveDate,UnderwriterName,UwrAssistantName,SpContractType,SpIsGapLessRenewal,FailReason
861,31492,Medical Information Technology,USE THIS ONE-Using last year's reports for pri...,In Underwriting,3,8,LOCK,01/01/2022,"Smith, Anna","Bhovar, Manish",1224,True,Gapless Renewal with an Incorrect Contract Type
1866,27884,Icahn Enterprises (ICAHN),500k 24/12,Declined,1,3,OPEN,01/01/2022,"Nickolas, David","Bates, Erick",2412,True,Gapless Renewal with an Incorrect Contract Type
2043,17954,"Appalachian Regional Healthcare System, Inc.","300K, 24/12",Declined,1,1,OPEN,01/01/2022,"Nickolas, David","Subramani, Santhosh",2412,True,Gapless Renewal with an Incorrect Contract Type
2284,9850,Zenith Insurance Company,Updated - NNL/RC50,Quoted,2,3,OPEN,07/01/2022,"Herda, Luke","Subramani, Santhosh",1224,True,Gapless Renewal with an Incorrect Contract Type
2850,9850,Zenith Insurance Company,Updated - NNL/RC50,Quoted,2,5,OPEN,07/01/2022,"Herda, Luke","Subramani, Santhosh",1224,True,Gapless Renewal with an Incorrect Contract Type
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49058,7393,MXR The Imaging Solution Company,RBP 6Degrees,Quoted,6,1,LOCK,09/01/2022,"Manning, Alexandra","Subramani, Santhosh",1224,True,Gapless Renewal with an Incorrect Contract Type
50511,11890,Kaweah Delta Health Care District,Current Year,In Underwriting,2,1,LOCK,01/01/2022,"Manning, Alexandra","Herda, Luke",1224,True,Gapless Renewal with an Incorrect Contract Type
52169,27276,"Tex-Tech Industries, Inc.",125K 24/12,Declined,1,1,OPEN,01/01/2022,"Kusiak, Andrew","Swiney, Toral",2412,True,Gapless Renewal with an Incorrect Contract Type
52816,7393,MXR The Imaging Solution Company,RBP 12.31,Quoted,7,2,LOCK,12/31/2022,"Manning, Alexandra","Subramani, Santhosh",1224,True,Gapless Renewal with an Incorrect Contract Type


# ESL Data Check  #7

- Premium Refund =  Yes
- Spec manual Workup equals 1.0

In [11]:
# -- Pull in Columns for Test -- #
df7 = df1[(df1['SpPremiumRefund'].isnull() == False)]
df7['SpPremiumRefund']=df7['SpPremiumRefund'].astype(int)
df7_con = df1[(df1['SpIsPremiumRefund']==True)& (df1['SpPremiumRefund']==1)][['AccountID',
              'AccountName','Description','QuoteStatusDescription','Scenario','OptionSequence',
              'Status','EffectiveDate','UnderwriterName','UwrAssistantName','SpPremiumRefund',
              'SpRefundSharePercent', 'SpLossRatio', 'SpCapPercent','SpIsNetCommission']].drop_duplicates()
df7_con['FailReason'] = 'Gapless Renewal with an Incorrect Contract Type'
# -- Write to File -- #
df7_con.to_csv('PremiumRefund7.csv')
with open('PremiumRefund7.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #7: Premium Refund = "Yes" and Spec Manual Workup = "1.0"' + '\n' +'\n' + content)

df7_con

<ipython-input-11-1bdfb8af800b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7['SpPremiumRefund']=df7['SpPremiumRefund'].astype(int)


,AccountID,AccountName,Description,QuoteStatusDescription,Scenario,OptionSequence,Status,EffectiveDate,UnderwriterName,UwrAssistantName,SpPremiumRefund,SpRefundSharePercent,SpLossRatio,SpCapPercent,SpIsNetCommission,FailReason
145,35778,Laseraway,Final Revision,Quoted,4,2,LOCK,06/01/2022,"Manning, Alexandra","Curry, Morgan",1.0,100.0,35.0,18.0,True,Gapless Renewal with an Incorrect Contract Type
196,22598,"Pipeline East Dallas, LLC","Combined - Pipeline East, SRC, Avanti",In Underwriting,2,3,OPEN,01/01/2022,"Manning, Alexandra","Curry, Morgan",1.0,25.0,65.0,15.0,False,Gapless Renewal with an Incorrect Contract Type
529,35778,Laseraway,Final Revision,Quoted,3,3,OPEN,06/01/2022,"Manning, Alexandra","Curry, Morgan",1.0,100.0,35.0,18.0,True,Gapless Renewal with an Incorrect Contract Type
627,35588,Project Lakehouse,"$100k - Firm, add 10% comm",Quoted,4,6,SOLD,04/01/2022,"Onufer, Max","Curry, Morgan",1.0,25.0,60.0,15.0,True,Gapless Renewal with an Incorrect Contract Type
681,22598,"Pipeline East Dallas, LLC","Combined - Pipeline East, SRC, Avanti",In Underwriting,2,1,OPEN,01/01/2022,"Manning, Alexandra","Curry, Morgan",1.0,25.0,65.0,15.0,False,Gapless Renewal with an Incorrect Contract Type
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53102,35588,Project Lakehouse,$100k - 9/9 and 9/12,In Underwriting,1,1,LOCK,04/01/2022,"Onufer, Max","Curry, Morgan",1.0,25.0,60.0,15.0,True,Gapless Renewal with an Incorrect Contract Type
53156,17559,Lawrence General Hospital,Renewal Scenario,Quoted,0,3,OPEN,01/01/2022,"Bohler, Abigail","Gunderson, Corey",1.0,25.0,65.0,15.0,False,Gapless Renewal with an Incorrect Contract Type
53273,35778,Laseraway,Final Revision,Quoted,3,2,OPEN,06/01/2022,"Manning, Alexandra","Curry, Morgan",1.0,100.0,35.0,18.0,True,Gapless Renewal with an Incorrect Contract Type
53326,22310,"Upstream Newco, Inc.",Current Year,In Underwriting,2,2,LOCK,01/01/2022,"Nanson, Jonathan","Bohler, Abigail",1.0,65.0,25.0,15.0,True,Gapless Renewal with an Incorrect Contract Type


# ESL Data Check  #8

- "Compensation" > 15.00


In [12]:
# -- Pull in Columns for Test -- #

df8_con = df1[(df1['EffectiveDate']>'01/31/2022') & (df1['SpCommission']>15)][['AccountID',
              'AccountName','Description','QuoteStatusDescription','Scenario','OptionSequence',
              'Status','EffectiveDate','UnderwriterName','UwrAssistantName','SpCommission']].drop_duplicates()
df8_con['FailReason'] = 'Compensation is higher than 15%'
# -- Write to File -- #
df8_con.to_csv('Compensation8.csv')
with open('Compensation8.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #8: Compensation greater than 15%' + '\n' +'\n' + content)
df8_con

,AccountID,AccountName,Description,QuoteStatusDescription,Scenario,OptionSequence,Status,EffectiveDate,UnderwriterName,UwrAssistantName,SpCommission,FailReason
5987,6335,Public Risk Management of Florida,Current Year,In Underwriting,2,2,LOCK,10/01/2022,"Gross, Jennifer","Bohler, Abigail",20.5,Compensation is higher than 15%
6456,6335,Public Risk Management of Florida,CY - old plan,Quoted,1,1,OPEN,10/01/2022,"Gross, Jennifer","Simon, Abigail",20.5,Compensation is higher than 15%
8381,35558,"U.S. Vision, Inc.","300000, 24/12",Declined,1,2,OPEN,04/01/2022,"Nickolas, David","Winnekins, Jackie",20.0,Compensation is higher than 15%
8826,6335,Public Risk Management of Florida,Updated April data,Quoted,3,2,LOCK,10/01/2022,"Gross, Jennifer","Bohler, Abigail",20.5,Compensation is higher than 15%
9051,6335,Public Risk Management of Florida,May Data - firm requested,Quoted,4,2,LOCK,10/01/2022,"Gross, Jennifer","Simon, Abigail",20.5,Compensation is higher than 15%
9080,6335,Public Risk Management of Florida,May Data - firm requested,Quoted,4,1,LOCK,10/01/2022,"Gross, Jennifer","Simon, Abigail",20.5,Compensation is higher than 15%
13677,6335,Public Risk Management of Florida,May Data - firm requested,Quoted,4,3,LOCK,10/01/2022,"Gross, Jennifer","Simon, Abigail",20.5,Compensation is higher than 15%
16480,18326,Central Unified School District (CUSD),Renewal Scenario,Quoted,0,1,OPEN,12/01/2021,NaN,NaN,18.0,Compensation is higher than 15%
19767,6335,Public Risk Management of Florida,Current Year,In Underwriting,2,3,LOCK,10/01/2022,"Gross, Jennifer","Bohler, Abigail",20.5,Compensation is higher than 15%
21160,35558,"U.S. Vision, Inc.","300000, 24/12",Declined,1,3,OPEN,04/01/2022,"Nickolas, David","Winnekins, Jackie",20.0,Compensation is higher than 15%


# ESL Data Check  #9

 - PPO Census Count <> Census

In [13]:
# -- Read Separate PPO File  ---#
PPO = pd.read_csv('PPO.csv', engine ='python')

# -- Pull in Columns for Test -- #
PPOdf = PPO[['PPOSequence','Census','AccountID','AccountName','Description','MostLikelyProposal','QuoteStatusDescription','IsRenewal','Status',
             'Scenario','Enrollment','EffectiveDate','UnderwriterName','UwrAssistantName']].drop_duplicates()

# -- Sum enrollment
PPOCensus=PPOdf.groupby(['AccountID','AccountName','Description','QuoteStatusDescription',
                         'EffectiveDate','Scenario','UnderwriterName','UwrAssistantName','Census']).agg({'Enrollment':'sum'})

#--Scenario 0--#
PPORdf = PPOdf[['PPOSequence','Census','AccountID','AccountName','Description','QuoteStatusDescription','Status','IsRenewal',
             'Scenario','MostLikelyProposal','Enrollment','EffectiveDate','UnderwriterName','UwrAssistantName']] 
PPORdf.loc[(PPORdf['IsRenewal'] == 1) & (PPORdf['Scenario'] == '0'), 'Compare'] = 'Yes'
PPORdf.loc[(PPORdf['IsRenewal'] == 1) & (PPORdf['Scenario'] == '1'), 'Compare'] = 'Yes'
PPORdf.loc[(PPORdf['IsRenewal'] == 1) & (PPORdf['Scenario'] == '2') & (PPORdf['Status'] == 'OPEN'), 'Compare'] = 'Yes'
PPORdf.loc[(PPORdf['IsRenewal'] == 1) & (PPORdf['MostLikelyProposal'] == 'Yes'), 'Compare'] = 'Yes'
PPORdf.loc[(PPORdf['Compare'].isnull()), 'Compare'] = 'No'
PPORdf=PPORdf[(PPORdf['Compare'] == 'Yes')]
PPORdf=PPORdf[['PPOSequence','Census','AccountID','AccountName','Description','QuoteStatusDescription','Status','IsRenewal',
             'Scenario','MostLikelyProposal','Enrollment','EffectiveDate','UnderwriterName','UwrAssistantName']] 
#--Scenario 1--#
PPONBdf = PPOdf[['PPOSequence','Census','AccountID','AccountName','Description','QuoteStatusDescription','Status','IsRenewal',
             'Scenario','MostLikelyProposal','Enrollment','EffectiveDate','UnderwriterName','UwrAssistantName']] 
PPONBdf=PPONBdf[(PPONBdf['IsRenewal'] == '0') & (PPONBdf['MostLikelyProposal'] == 'Yes')]

PPORdf

C:\Users\i706522\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\i706522\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\i706522\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

,PPOSequence,Census,AccountID,AccountName,Description,QuoteStatusDescription,Status,IsRenewal,Scenario,MostLikelyProposal,Enrollment,EffectiveDate,UnderwriterName,UwrAssistantName
13,1,2608,11396,Sidley Austin LLP,Sold,Quoted,SOLD,1,5,Yes,2608,01/01/2022,"Eichler, Mark","Lee, Pa"
21,1,187,5434,Heat and Frost Insulators of Northern Californ...,"Sold renewal $325,000",Quoted,SOLD,1,3,Yes,187,01/01/2022,"Brandsness, Eric","Herda, Luke"
26,2,408,7078,Grand River Hospital District,updated,Quoted,LOCK,1,3,Yes,1,01/01/2022,"Brandsness, Eric","Herda, Luke"
45,1,434,5653,"St. Norbert College, Inc.",SOLD RENEWAL,Quoted,SOLD,1,3,Yes,434,01/01/2022,"Smith, Anna","Herda, Luke"
59,1,1214,9942,"WW International, Inc.",SOLD Renewal $400K,Quoted,SOLD,1,6,Yes,1214,01/01/2022,"Kusiak, Andrew","Herda, Luke"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21145,1,513,22509,"Clay Electric Cooperative, Inc.",final,Quoted,LOCK,1,6,Yes,513,01/01/2022,"Gross, Jennifer","Herda, Luke"
21186,1,612,9669,"BayCare Health Systems, L.L.C.",Sold renewal Utopia Checkout,Quoted,SOLD,1,4,Yes,612,01/01/2022,"Smith, Anna","Lee, Pa"
21203,1,299,9678,NatWest Markets Securities Inc.,Sold Renewal $400K,Quoted,SOLD,1,4,Yes,299,01/01/2022,"Kusiak, Andrew","Garrod, Quinn"
21205,1,435,15223,"TIDI Products, LLC",FINAL SOLD USE THIS ONE,Quoted,SOLD,1,6,Yes,435,01/01/2022,"Smith, Anna","Lee, Pa"


In [14]:
PPOCensus= pd.concat([PPONBdf, PPORdf])

# -- Write to file to remove double index -- #
PPOCensus.to_csv('PPOCENSUSCheck.csv')
PPOCensus= pd.read_csv('PPOCENSUSCheck.csv')

# -- Compare PPO Census total to Group Total -- #
PPOCensus['Census']=PPOCensus['Census'].astype(int)
PPOCensus['Check']=PPOCensus['Census'] - PPOCensus['Enrollment']
PPOCensus =PPOCensus[~(PPOCensus['Check']==0)]
PPOCensus['FailReason'] = 'PPO Life Total does not match the Census'
df9_con = PPOCensus
PPOCensus.to_csv('PPOCensusCount9.csv')
with open('PPOCensusCount9.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #9: PPO Life Count does not match the Census' + '\n' +'\n' + content)
PPOCensus
PPO

,UwrMDSLQuoteId,PPONetworkDescription,NetworkId,Enrollment,EffectiveDate,UwrMDSLScenarioId,AccountName,AccountID,UnderwriterName,UwrAssistantName,QuoteStatusDescription,IsRenewal,Description,Status,Census,MostLikelyProposal,Rater,PPOSequence,Scenario
0,91263,2022-01 United Healthcare Choice Plus,33501,207,07/01/2022,91263,United Heritage Credit Union,35675,"Bohler, Abigail",NaN,In Underwriting,0,"75000, 15/12",LOCK,207,No,2020v2,1,1
1,90836,2022-01 Aetna ASO,33497,479,04/01/2022,90836,Alliance Inspection Management,24746,"Garrod, Quinn","Curry, Morgan",Quoted,0,125% w/o TLO,OPEN,479,No,2020v2,1,3
2,73887,2022-01 United HealthCare,33500,2539,01/01/2022,73887,Brown Brothers Harriman & Co.,4937,"Nickolas, David","Her, Susan",Quoted,0,updated effective date and corrected manual,LOCK,2539,Yes,2020v2,1,2
3,74896,2022-01 BlueCross BlueShield (State),33498,315,01/01/2022,74896,Texas Biomedical Research Institute,33910,"Onufer, Max","Bates, Erick",In Underwriting,0,90k 24/12,LOCK,315,No,2020v2,1,1
4,82708,2022-01 BlueCross BlueShield (State),33498,382,01/01/2022,82708,"Bauer Built, Inc.",15959,"Hobbs, Christina","Kaping, Darren",Declined,0,"$175k, 24/12",OPEN,382,No,2020v2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21238,87406,2022-01 Cigna ASO,33499,10401,01/01/2022,87406,"J.B. Hunt Transport, Inc.",5104,"O'Donnell, Mary","Garrod, Quinn",Quoted,1,"firm, Sept. data",LOCK,19519,No,2020v2,1,4
21239,87649,2022-01 United Healthcare Choice Plus,33501,2465,01/01/2022,87649,CSAA Insurance Exchange,7590,"Manning, Alexandra","Subramani, Santhosh",In Underwriting,0,"500000, 24/12",LOCK,2465,No,2020v2,1,1
21240,80315,2022-01 BlueCross BlueShield (State),33498,1456,01/01/2022,80315,Gate Petroleum Company,28269,"Gross, Jennifer","Her, Susan",Declined,0,"450000, 24/12",OPEN,1456,No,2020v2,1,1
21241,80249,2022-01 Cigna ASO,33499,1608,01/01/2022,80249,Suez Water Resources Inc.,31642,"Smith, Anna","Mitchell, Zachary",Declined,0,"400000, 24/12",OPEN,1608,No,2020v2,1,1


# ESL Data Check  #10

 - Rate to rate column BM "Census" does not equal sum of column AC "Administrator 1 Lives", column AE "Administrator 2 Lives", Column AG "Administrator 3 Lives", AI "Administrator 4 Lives" and AK "Administrator 5 Lives" 




In [15]:
# -- Pull in Columns for Test -- #

dfacensus=df1[['AccountID','AccountName','Description','QuoteStatusDescription','Scenario','OptionSequence',
               'Census','Status','EffectiveDate','UnderwriterName','UwrAssistantName','ClaimAdministrator1',
               'ClaimAdministrator1Fee','ClaimAdministrator1Lives','ClaimAdministrator2','ClaimAdministrator2Fee',
               'ClaimAdministrator2Lives','ClaimAdministrator3','ClaimAdministrator3Fee','ClaimAdministrator3Lives',
               'ClaimAdministrator4','ClaimAdministrator4Fee','ClaimAdministrator4Lives','ClaimAdministrator5',
               'ClaimAdministrator5Fee','ClaimAdministrator5Lives']].drop_duplicates()


dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'UMR'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'UMR - Phoenix'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'UMR, formerly known as Fiserv-BeneSight'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'InforMed, LLC'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'Healthcare Management Administrators (HMA)'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'Harrington Health'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'NGS American'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'Principal Financial Group'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'PreferredOne Administrative Services'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'HealthSmart Benefit Solutions'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'Group & Pension Administrators, Inc.'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'FMH CoreSource'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'CoreSource, Inc.'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'CoreSource'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'Comprehensive Care Services'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator1'] == 'Auxiant'), 'KeyAdmin'] = 'Yes1'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'Allied Benefit Systems, Inc.'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'UMR'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'UMR - Phoenix'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'UMR, formerly known as Fiserv-BeneSight'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'InforMed, LLC'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'Healthcare Management Administrators (HMA)'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'Harrington Health'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'NGS American'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'Principal Financial Group'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'PreferredOne Administrative Services'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'HealthSmart Benefit Solutions'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'Group & Pension Administrators, Inc.'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'FMH CoreSource'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'CoreSource, Inc.'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'CoreSource'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'Comprehensive Care Services'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'Auxiant'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator2'] == 'Allied Benefit Systems, Inc.'), 'KeyAdmin'] = 'Yes2'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'Allied Benefit Systems, Inc.'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'UMR'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'UMR - Phoenix'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'UMR, formerly known as Fiserv-BeneSight'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'InforMed, LLC'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'Healthcare Management Administrators (HMA)'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'Harrington Health'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'NGS American'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'Principal Financial Group'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'PreferredOne Administrative Services'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'HealthSmart Benefit Solutions'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'Group & Pension Administrators, Inc.'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'FMH CoreSource'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'CoreSource, Inc.'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'CoreSource'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'Comprehensive Care Services'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'Auxiant'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator3'] == 'Allied Benefit Systems, Inc.'), 'KeyAdmin'] = 'Yes3'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'Allied Benefit Systems, Inc.'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'UMR'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'UMR - Phoenix'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'UMR, formerly known as Fiserv-BeneSight'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'InforMed, LLC'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'Healthcare Management Administrators (HMA)'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'Harrington Health'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'NGS American'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'Principal Financial Group'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'PreferredOne Administrative Services'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'HealthSmart Benefit Solutions'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'Group & Pension Administrators, Inc.'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'FMH CoreSource'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'CoreSource, Inc.'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'CoreSource'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'Comprehensive Care Services'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'Auxiant'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator4'] == 'Allied Benefit Systems, Inc.'), 'KeyAdmin'] = 'Yes4'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'Allied Benefit Systems, Inc.'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'UMR'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'UMR - Phoenix'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'UMR, formerly known as Fiserv-BeneSight'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'InforMed, LLC'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'Healthcare Management Administrators (HMA)'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'Harrington Health'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'NGS American'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'Principal Financial Group'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'PreferredOne Administrative Services'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'HealthSmart Benefit Solutions'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'Group & Pension Administrators, Inc.'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'FMH CoreSource'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'CoreSource, Inc.'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'CoreSource'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'Comprehensive Care Services'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'Auxiant'), 'KeyAdmin'] = 'Yes5'
dfacensus.loc[(dfacensus['ClaimAdministrator5'] == 'Allied Benefit Systems, Inc.'), 'KeyAdmin'] = 'Yes5'




In [16]:
# -- Filter for OptionSequence 1 and Scenario 1 and fill Null rows --#
#dfacensus=dfacensus[(dfacensus['OptionSequence'] == 1) & (dfacensus['KeyAdmin'] == 'Yes')]



# -- Total Administrator fees --#
# dfacensus['AdminFeetotal']=dfacensus['ClaimAdministrator1Fee']+dfacensus['ClaimAdministrator2Fee']+dfacensus['ClaimAdministrator3Fee']+dfacensus['ClaimAdministrator4Fee']

# -- Total Administrator census --#
dfacensus['AdminCensusTotal']=dfacensus['ClaimAdministrator1Lives']+dfacensus['ClaimAdministrator2Lives']+dfacensus['ClaimAdministrator3Lives']+dfacensus['ClaimAdministrator4Lives']+dfacensus['ClaimAdministrator5Lives']
   
# -- Compare Administrator Census to Group Census --#  
dfacensus['AdminDifference']=dfacensus['Census']-dfacensus['AdminCensusTotal']
dfacensus =dfacensus[~(dfacensus['AdminDifference']==0) & (dfacensus['OptionSequence']==1)]
dfacensus.to_csv('Admincheck.csv') 

dfa1=dfacensus[['AccountID','AccountName','Description','QuoteStatusDescription','Scenario','OptionSequence',
               'Census','Status','EffectiveDate','UnderwriterName','UwrAssistantName','ClaimAdministrator1',
               'ClaimAdministrator1Fee','ClaimAdministrator1Lives','KeyAdmin','AdminDifference']]
dfa1=dfa1[(dfa1['KeyAdmin']=='Yes1')]

dfa2=dfacensus[['AccountID','AccountName','Description','QuoteStatusDescription','Scenario','OptionSequence',
               'Census','Status','EffectiveDate','UnderwriterName','UwrAssistantName','ClaimAdministrator2',
               'ClaimAdministrator2Fee','ClaimAdministrator2Lives','KeyAdmin','AdminDifference']]
dfa2=dfa2[(dfa2['KeyAdmin']=='Yes2')]

dfa3=dfacensus[['AccountID','AccountName','Description','QuoteStatusDescription','Scenario','OptionSequence',
               'Census','Status','EffectiveDate','UnderwriterName','UwrAssistantName','ClaimAdministrator3',
               'ClaimAdministrator3Fee','ClaimAdministrator3Lives','KeyAdmin','AdminDifference']]
dfa3=dfa3[(dfa3['KeyAdmin']=='Yes3')]

dfa4=dfacensus[['AccountID','AccountName','Description','QuoteStatusDescription','Scenario','OptionSequence',
               'Census','Status','EffectiveDate','UnderwriterName','UwrAssistantName','ClaimAdministrator4',
               'ClaimAdministrator4Fee','ClaimAdministrator4Lives','KeyAdmin','AdminDifference']]
dfa4=dfa4[(dfa4['KeyAdmin']=='Yes4')]

dfa5=dfacensus[['AccountID','AccountName','Description','QuoteStatusDescription','Scenario','OptionSequence',
               'Census','Status','EffectiveDate','UnderwriterName','UwrAssistantName','ClaimAdministrator5',
               'ClaimAdministrator5Fee','ClaimAdministrator5Lives','KeyAdmin','AdminDifference']]
dfa5=dfa5[(dfa5['KeyAdmin']=='Yes5')]

dfa1 = dfa1.append([dfa2,dfa3,dfa4,dfa5])
dfa1['FailReason'] = 'Administrator Census total does not match the full census'
df10_con = dfa1



In [17]:
# -- Compare Administrator Census to Group Census --#  
dfa1.to_csv('AdministratorCensus10.csv')

with open('AdministratorCensus10.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #10: Administrator Life Count does not match the Census - Fee included)' + '\n' +'\n' + content)
dfa1.to_csv('AdminCensusfeeIncluded.csv')

# ESL Data Check #11

In [18]:
#-- Pull Columns --#
df11_con = df1[['AccountID','AccountName','Description','QuoteStatusDescription','Scenario',
                'OptionSequence','Status','EffectiveDate','UnderwriterName','UwrAssistantName',
                'RenewalTier','MostLikelyProposal','IsNumberOfSpouseChildrenOverride',
                'UwrMDSLScenarioId']].drop_duplicates()

# -- Filter for Renewals --#
df11_con = df11_con[(df11_con['OptionSequence'] == 1) & (df11_con['RenewalTier'] == 'UNKN')]
S0= df11_con[['AccountID','AccountName','UnderwriterName','UwrAssistantName','QuoteStatusDescription','Description',
              'RenewalTier','Status','EffectiveDate','IsNumberOfSpouseChildrenOverride','Scenario','OptionSequence']]

#--Scenario 0--#
S0= S0[(S0['Scenario'] == 0)]
S0.rename(columns={'IsNumberOfSpouseChildrenOverride':'S0Override','Scenario':'Scenario0'}, inplace = True)


#--Scenario 1--#
S1= df11_con[['AccountID','IsNumberOfSpouseChildrenOverride','Scenario']]
S1.to_csv('Scenario1test.csv')
S1= S1[(S1['Scenario'] == 1)]
S1.rename(columns={'IsNumberOfSpouseChildrenOverride':'S1Override','Scenario':'Scenario1'}, inplace = True)


#-- Most Likely Scenario --#
ML= df11_con[(df11_con['MostLikelyProposal'] == 'Yes')]
ML= ML[['AccountID','IsNumberOfSpouseChildrenOverride','Scenario']]
ML.rename(columns={'IsNumberOfSpouseChildrenOverride':'MLOverride','Scenario':'MLScenario'}, inplace = True)


#-- Sold Scenario --#
S= df11_con[(df11_con['Status'] == 'SOLD')]
S= S[['AccountID','IsNumberOfSpouseChildrenOverride','Scenario']]
S.rename(columns={'IsNumberOfSpouseChildrenOverride':'SoldOverride','Scenario':'SoldScenario'}, inplace = True)


# -- Merge filtered Scenarios above using Accounts ID --#
dfs = [S0, S1, ML, S]
SCOverride = reduce(lambda  left,right: pd.merge(left,right,on=['AccountID'],
                                            how='left'), dfs).fillna('-1')

# --- Convert Scenarios to Integers --- #
convert_dict = {'Scenario0': int,
               'Scenario1': int,
               'MLScenario': int,
               'SoldScenario': int
               }  
SCOverride = SCOverride.astype(convert_dict)  

SCOverride

,AccountID,AccountName,UnderwriterName,UwrAssistantName,QuoteStatusDescription,Description,RenewalTier,Status,EffectiveDate,S0Override,Scenario0,OptionSequence,S1Override,Scenario1,MLOverride,MLScenario,SoldOverride,SoldScenario
0,23263,"Main Event Entertainment, Inc.","Onufer, Max","Herda, Luke",In Underwriting,Renewal Scenario,UNKN,OPEN,01/01/2022,False,0,1,False,1,False,3,-1,-1
1,19276,Fiesta Tableware Company,"Aiello, Rhonda","Garrod, Quinn",Quoted,Renewal Scenario,UNKN,OPEN,06/01/2022,True,0,1,True,1,True,6,True,6
2,10091,"Pernod Ricard USA, LLC","Nickolas, David","Garrod, Quinn",Quoted,Renewal Scenario,UNKN,OPEN,01/01/2022,False,0,1,False,1,False,8,False,8
3,32187,Grant County Public Hospital District 1 dba Sa...,"Brandsness, Eric","Herda, Luke",Quoted,Renewal Scenario,UNKN,OPEN,01/01/2022,True,0,1,True,1,True,4,-1,-1
4,32380,SRTV Holdings LLC,"Gross, Jennifer",-1,Quoted,Renewal Scenario,UNKN,OPEN,01/01/2022,False,0,1,False,1,False,7,False,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,4661,I.A.T.S.E. Local 16 Health & Welfare Trust Fund,"Manning, Alexandra","Herda, Luke",Quoted,Renewal Scenario,UNKN,OPEN,01/01/2022,False,0,1,False,1,False,5,False,5
842,18853,Howell Township Board of Education,"Tollis, Elizabeth","Lee, Pa",Quoted,Renewal Scenario,UNKN,OPEN,07/01/2022,True,0,1,-1,-1,True,9,True,9
843,5794,"Arkema, Inc.","Gross, Jennifer","Lee, Pa",Quoted,Renewal Scenario,UNKN,OPEN,01/01/2022,False,0,1,-1,-1,False,7,False,7
844,9971,Zekelman Industries,"Eichler, Mark","Lee, Pa",In Underwriting,Renewal Scenario,UNKN,OPEN,01/01/2022,False,0,1,False,1,False,5,False,5


In [19]:
#-- Test Files for Filtered Tables above --#
S0.to_csv('Scenario0.csv')
#S1.to_csv('Scenario1.csv')
#ML.to_csv('Mostlikely.csv')
#S.to_csv('Sold.csv')
#SCOverride.to_csv('SCOverride.csv')

# -- Test for Correct DataTypes -- #
display(SCOverride.dtypes)

AccountID                  int64
AccountName               object
UnderwriterName           object
UwrAssistantName          object
QuoteStatusDescription    object
Description               object
RenewalTier               object
Status                    object
EffectiveDate             object
S0Override                  bool
Scenario0                  int32
OptionSequence             int64
S1Override                object
Scenario1                  int32
MLOverride                object
MLScenario                 int32
SoldOverride              object
SoldScenario               int32
dtype: object

In [20]:
# --- Test for Spouse/ Child Match for Scenario 0&1, Scenario0 & Most Likely, Scenario0 and Sold ---#

#---- Does Scenario Spouse and Child Override Match for Scenario 0 and Scenario 1 ---#
SCOverride.loc[(SCOverride['Scenario1'] > 0) & (SCOverride['S1Override'] != SCOverride['S0Override'])
               , 'Scenario 0 - 1 Match'] = 'Fail' 

SCOverride.loc[(SCOverride['Scenario1'] > 0) & (SCOverride['S1Override'] == SCOverride['S0Override'])
               , 'Scenario 0 - 1 Match'] = 'Pass'

SCOverride.loc[(SCOverride['Scenario1'] == -1), 'Scenario 0 - 1 Match'] = 'N/A' 


#---- Does Scenario Spouse and Child Override Match for Scenario 0 and Sold Scenario ---#

SCOverride.loc[(SCOverride['SoldScenario'] > 0) & (SCOverride['SoldOverride'] != SCOverride['S0Override'])
               , 'Sold Match'] = 'Fail' 

SCOverride.loc[(SCOverride['SoldScenario'] > 0) & (SCOverride['SoldOverride'] == SCOverride['S0Override'])
               , 'Sold Match'] = 'Pass'

SCOverride.loc[(SCOverride['SoldScenario'] == -1), 'Sold Match'] = 'N/A' 


#---- Does Scenario Spouse and Child Override Match for Scenario 0 and Most likely Scenario ---#
SCOverride.loc[(SCOverride['MLScenario'] > 0) & (SCOverride['MLOverride'] != SCOverride['S0Override'])
               , 'MostLikely Match'] = 'Fail'  

SCOverride.loc[(SCOverride['MLScenario'] >= 0) & (SCOverride['MLOverride'] == SCOverride['S0Override'])
               , 'MostLikely Match'] = 'Pass'

SCOverride.loc[(SCOverride['MLScenario'] == -1), 'MostLikely Match'] = 'N/A'



# --- Test for Overall Result---#
#--- Test 1 Scenario 0 - 1 do not match---#
SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'Fail'), 'Result'] = 'Fail'



#--- Test 2 Scenario 0  and Most Likely match ?---#

SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'Pass') & (SCOverride['Sold Match'] == 'N/A') 
               & (SCOverride['MostLikely Match'] == 'Pass'), 'Result'] = 'Pass'

SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'Pass') & (SCOverride['Sold Match'] == 'N/A') 
               & (SCOverride['MostLikely Match'] == 'Fail'), 'Result'] = 'Fail'


#--- Test 2 Scenario 0, Most Likely, and Sold match ?---#

SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'Pass') & (SCOverride['Sold Match'] == 'Fail') 
               & (SCOverride['MostLikely Match'] == 'Pass'), 'Result'] = 'Fail'

SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'Pass') & (SCOverride['Sold Match'] == 'Fail') 
               & (SCOverride['MostLikely Match'] == 'Fail'), 'Result'] = 'Fail'

SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'Pass') & (SCOverride['Sold Match'] == 'Pass') 
               & (SCOverride['MostLikely Match'] == 'Fail'), 'Result'] = 'Fail'

SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'Pass') & (SCOverride['Sold Match'] == 'Pass') 
               & (SCOverride['MostLikely Match'] == 'Pass'), 'Result'] = 'Pass'


#--- Test 3 Scenario 0, Most Likely, and Sold match ?---#
SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'Pass') & (SCOverride['MostLikely Match'] == 'Fail'), 'Result'] = 'Fail' 
SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'Pass') & (SCOverride['MostLikely Match'] == 'N/A'), 'Result'] = 'Pass' 
SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'N/A')  & (SCOverride['MostLikely Match'] == 'Pass'), 'Result'] = 'Pass'

#--Pass Scenario 1 groups not included in the matches above---#

SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'Pass') 
                & (SCOverride['Result'].isnull()), 'Result'] = 'Pass'  

SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'N/A') & (SCOverride['Sold Match'] == 'N/A') 
                & (SCOverride['MostLikely Match'].isnull()), 'Result'] = 'Pass'  

SCOverride.loc[(SCOverride['Scenario 0 - 1 Match'] == 'N/A') & (SCOverride['Sold Match'] == 'N/A') 
                & (SCOverride['MostLikely Match'] == 'N/A'), 'Result'] = 'Pass'    


#---Filter Final Test Results ---#
SCOverride=SCOverride[['AccountID','AccountName','UnderwriterName','UwrAssistantName','QuoteStatusDescription','RenewalTier',
                       'Status','EffectiveDate','S0Override','Scenario0','S1Override','Scenario1','MLOverride','MLScenario',
                       'SoldOverride','SoldScenario','Scenario 0 - 1 Match','Sold Match','MostLikely Match','Result']].drop_duplicates()
SCOverride=SCOverride[((SCOverride['QuoteStatusDescription'] == 'Declined')|(SCOverride['QuoteStatusDescription'] == 'Quoted')
                       |(SCOverride['QuoteStatusDescription'] == 'Quoted Dead')|(SCOverride['QuoteStatusDescription'] == 'In Underwriting')
                       |(SCOverride['QuoteStatusDescription'] == 'Pended')|(SCOverride['QuoteStatusDescription'] == 'Sold'))]





In [21]:
#--Clean up results --#

SCOverride.replace({
                True: 'Yes',
                False: 'No'
            })
convert_dict = {'Scenario0': str,
               'Scenario1': str,
               'MLScenario': str,
               'SoldScenario': str,
                'S0Override': str,
                'S1Override': str,
                'SoldOverride': str,
                'MLOverride': str,
               }  
SCOverride = SCOverride.astype(convert_dict) 



In [22]:
SCOverride['Scenario0']=SCOverride['Scenario0'].str.replace('-1','')
SCOverride['Scenario1']=SCOverride['Scenario1'].str.replace('-1','')
SCOverride['MLScenario']=SCOverride['MLScenario'].str.replace('-1','')
SCOverride['SoldScenario']=SCOverride['SoldScenario'].str.replace('-1','')
SCOverride['S0Override']=SCOverride['S0Override'].str.replace('-1','')
SCOverride['S0Override']=SCOverride['S0Override'].str.replace('True','Yes')
SCOverride['S0Override']=SCOverride['S0Override'].str.replace('False','No')
SCOverride['S1Override']=SCOverride['S1Override'].str.replace('-1','')
SCOverride['S1Override']=SCOverride['S1Override'].str.replace('True','Yes')
SCOverride['S1Override']=SCOverride['S1Override'].str.replace('False','No')
SCOverride['MLOverride']=SCOverride['MLOverride'].str.replace('-1','')
SCOverride['MLOverride']=SCOverride['MLOverride'].str.replace('True','Yes')
SCOverride['MLOverride']=SCOverride['MLOverride'].str.replace('False','No')
SCOverride['SoldOverride']=SCOverride['SoldOverride'].str.replace('-1','')
SCOverride['SoldOverride']=SCOverride['SoldOverride'].str.replace('True','Yes')
SCOverride['SoldOverride']=SCOverride['SoldOverride'].str.replace('False','No')
SCOverride.to_csv('SpouseandChildOverride11.csv')    
SCOverride['FailReason']='Spouse and Child Override is not consistent' 
SCOverride['Scenario']='N/A'
SCOverride['OptionSequence']='1'
df11_con = SCOverride
with open('SpouseandChildOverride11.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #11: Spouse and Child Override matches for Scenario 0, 1, Most Likely, and Sold' + '\n' +'\n' + content)
SCOverride

,AccountID,AccountName,UnderwriterName,UwrAssistantName,QuoteStatusDescription,RenewalTier,Status,EffectiveDate,S0Override,Scenario0,...,MLScenario,SoldOverride,SoldScenario,Scenario 0 - 1 Match,Sold Match,MostLikely Match,Result,FailReason,Scenario,OptionSequence
0,23263,"Main Event Entertainment, Inc.","Onufer, Max","Herda, Luke",In Underwriting,UNKN,OPEN,01/01/2022,No,0,...,3,,,Pass,N/A,Pass,Pass,Spouse and Child Override is not consistent,N/A,1
1,19276,Fiesta Tableware Company,"Aiello, Rhonda","Garrod, Quinn",Quoted,UNKN,OPEN,06/01/2022,Yes,0,...,6,Yes,6,Pass,Pass,Pass,Pass,Spouse and Child Override is not consistent,N/A,1
2,10091,"Pernod Ricard USA, LLC","Nickolas, David","Garrod, Quinn",Quoted,UNKN,OPEN,01/01/2022,No,0,...,8,No,8,Pass,Pass,Pass,Pass,Spouse and Child Override is not consistent,N/A,1
3,32187,Grant County Public Hospital District 1 dba Sa...,"Brandsness, Eric","Herda, Luke",Quoted,UNKN,OPEN,01/01/2022,Yes,0,...,4,,,Pass,N/A,Pass,Pass,Spouse and Child Override is not consistent,N/A,1
4,32380,SRTV Holdings LLC,"Gross, Jennifer",-1,Quoted,UNKN,OPEN,01/01/2022,No,0,...,7,No,7,Pass,Pass,Pass,Pass,Spouse and Child Override is not consistent,N/A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,4661,I.A.T.S.E. Local 16 Health & Welfare Trust Fund,"Manning, Alexandra","Herda, Luke",Quoted,UNKN,OPEN,01/01/2022,No,0,...,5,No,5,Pass,Pass,Pass,Pass,Spouse and Child Override is not consistent,N/A,1
842,18853,Howell Township Board of Education,"Tollis, Elizabeth","Lee, Pa",Quoted,UNKN,OPEN,07/01/2022,Yes,0,...,9,Yes,9,N/A,Pass,Pass,Pass,Spouse and Child Override is not consistent,N/A,1
843,5794,"Arkema, Inc.","Gross, Jennifer","Lee, Pa",Quoted,UNKN,OPEN,01/01/2022,No,0,...,7,No,7,N/A,Pass,Pass,Pass,Spouse and Child Override is not consistent,N/A,1
844,9971,Zekelman Industries,"Eichler, Mark","Lee, Pa",In Underwriting,UNKN,OPEN,01/01/2022,No,0,...,5,No,5,Pass,Pass,Pass,Pass,Spouse and Child Override is not consistent,N/A,1


# ESL Data Check #12

In [23]:
#-- Pull Columns --#

df12_con = df1[['AccountID','AccountName','Description','QuoteStatusDescription','Scenario',
                'OptionSequence','Status','EffectiveDate','UnderwriterName','UwrAssistantName',
                'RenewalTier','MostLikelyProposal','CensusEE','CensusES','CensusEC','CensusEF',
                'UwrMDSLScenarioId']].drop_duplicates()

#--Filter for Renewal Cases--#

df12_con = df12_con[(df12_con['OptionSequence'] == 1) & (df12_con['RenewalTier'] == 'UNKN')]
df12_con.fillna(0)



#--Scenario 0--#
TierS0 = df12_con[['AccountID','AccountName','Description','QuoteStatusDescription',
                'OptionSequence','Status','EffectiveDate','UnderwriterName','UwrAssistantName',
                'RenewalTier','CensusEE','CensusES','CensusEC','CensusEF','Scenario','MostLikelyProposal']] 
TierS0=TierS0[(TierS0['Scenario'] == 0)]
TierS0.rename(columns={'CensusEE':'EES0','CensusES':'ESS0','CensusEC':'ECS0',
                   'CensusEF':'EFS0','Scenario':'Scenario 0'}, inplace = True)



#--Scenario 1--#
TierS1= df12_con[['AccountID','CensusEE','CensusES','CensusEC','CensusEF','Scenario']]
TierS1= TierS1[(TierS1['Scenario'] == 1)]
TierS1.rename(columns={'CensusEE':'EES1','CensusES':'ESS1','CensusEC':'ECS1',
                   'CensusEF':'EFS1','Scenario':'Scenario 1'}, inplace = True)


#--Most Likely Scenario--#
TierML= df12_con[(df12_con['MostLikelyProposal'] == 'Yes')]
TierML= TierML[['AccountID','CensusEE','CensusES','CensusEC','CensusEF','Scenario']]
TierML.rename(columns={'CensusEE':'EEML','CensusES':'ESML','CensusEC':'ECML',
                   'CensusEF':'EFML','Scenario':'Most Likely Scenario'}, inplace = True)


#--Sold Scenario--#
TierS= df12_con[(df12_con['Status'] == 'SOLD')]
TierS= TierS[['AccountID','CensusEE','CensusES','CensusEC','CensusEF','Scenario']]
TierS.rename(columns={'CensusEE':'EES','CensusES':'ESS','CensusEC':'ECS',
                   'CensusEF':'EFS','Scenario':'SoldScenario'}, inplace = True)


# -- Merge Filtered Tiers on Account ID -- #
dfs = [TierS0, TierS1, TierML, TierS]
Tier = reduce(lambda  left,right: pd.merge(left,right,on=['AccountID'],
                                            how='outer'), dfs).fillna('-1')




Tier

,AccountID,AccountName,Description,QuoteStatusDescription,OptionSequence,Status,EffectiveDate,UnderwriterName,UwrAssistantName,RenewalTier,...,EEML,ESML,ECML,EFML,Most Likely Scenario,EES,ESS,ECS,EFS,SoldScenario
0,23263,"Main Event Entertainment, Inc.",Renewal Scenario,In Underwriting,1,OPEN,01/01/2022,"Onufer, Max","Herda, Luke",UNKN,...,418,39,60,111,3,-1,-1,-1,-1,-1
1,19276,Fiesta Tableware Company,Renewal Scenario,Quoted,1,OPEN,06/01/2022,"Aiello, Rhonda","Garrod, Quinn",UNKN,...,151,0,0,174,6,151,0,0,174,6
2,10091,"Pernod Ricard USA, LLC",Renewal Scenario,Quoted,1,OPEN,01/01/2022,"Nickolas, David","Garrod, Quinn",UNKN,...,473,125,88,302,8,473,125,88,302,8
3,32187,Grant County Public Hospital District 1 dba Sa...,Renewal Scenario,Quoted,1,OPEN,01/01/2022,"Brandsness, Eric","Herda, Luke",UNKN,...,256,83,96,132,4,-1,-1,-1,-1,-1
4,32380,SRTV Holdings LLC,Renewal Scenario,Quoted,1,OPEN,01/01/2022,"Gross, Jennifer",-1,UNKN,...,148,36,56,57,7,148,36,56,57,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,6464,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,2883,763,868,1297,4,2883,763,868,1297,4
919,7358,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,589,245,0,774,4,589,245,0,774,4
920,11630,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,427,105,121,90,5,-1,-1,-1,-1,-1
921,31128,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,212,14,60,45,4,212,14,60,45,4


In [24]:
#-- Filter Files for Troubleshooting --#
#TierS0.to_csv('TierScenario0.csv')
#TierS1.to_csv('TierScenario1.csv')
#TierML.to_csv('TierMostlikely.csv')
#TierS.to_csv('Tiersold.csv')
#Tier.to_csv('Test12.csv')

#--Convert Tiered Census to Integer --#
Tier['EES0']=Tier['EES0'].astype(int)
Tier['ESS0']=Tier['ESS0'].astype(int)
Tier['ECS0']=Tier['ECS0'].astype(int)
Tier['EFS0']=Tier['EFS0'].astype(int)
Tier['EES1']=Tier['EES1'].astype(int)
Tier['ESS1']=Tier['ESS1'].astype(int)
Tier['ECS1']=Tier['ECS1'].astype(int)
Tier['EFS1']=Tier['EFS1'].astype(int)
Tier['EEML']=Tier['EEML'].astype(int)
Tier['ESML']=Tier['ESML'].astype(int)
Tier['ECML']=Tier['ECML'].astype(int)
Tier['EFML']=Tier['EFML'].astype(int)
Tier['EES']=Tier['EES'].astype(int)
Tier['ESS']=Tier['ESS'].astype(int)
Tier['ECS']=Tier['ECS'].astype(int)
Tier['EFS']=Tier['EFS'].astype(int)
display(Tier.dtypes)


AccountID                  int64
AccountName               object
Description               object
QuoteStatusDescription    object
OptionSequence            object
Status                    object
EffectiveDate             object
UnderwriterName           object
UwrAssistantName          object
RenewalTier               object
EES0                       int32
ESS0                       int32
ECS0                       int32
EFS0                       int32
Scenario 0                object
MostLikelyProposal        object
EES1                       int32
ESS1                       int32
ECS1                       int32
EFS1                       int32
Scenario 1                object
EEML                       int32
ESML                       int32
ECML                       int32
EFML                       int32
Most Likely Scenario      object
EES                        int32
ESS                        int32
ECS                        int32
EFS                        int32
SoldScenar

In [25]:
# -- Tiered Breakout --#
# Note: Test does not match census count it checks for consistent tier selection across Scenarios #

# -- Scenario 0 -- #
Tier.loc[(Tier['EES0'] != -1) & (Tier['EES0'] > 0) & (Tier['ESS0'] > 0) & (Tier['ECS0'] > 0) & (Tier['EFS0'] > 0), 'Scenario 0 Tiers'] = '4 Tier' # All ON1
Tier.loc[(Tier['EES0'] != -1) & (Tier['EES0'] > 0) & (Tier['ESS0'] == 0) & (Tier['ECS0'] == 0) & (Tier['EFS0'] > 0), 'Scenario 0 Tiers'] = 'EE + Family'
Tier.loc[(Tier['EES0'] != -1) & (Tier['EES0'] > 0) & (Tier['ESS0'] > 0) & (Tier['ECS0'] == 0), 'Scenario 0 Tiers'] = 'EE + SP'
Tier.loc[(Tier['EES0'] != -1) & (Tier['EES0'] > 0) & (Tier['ESS0'] == 0) & (Tier['ECS0'] > 0), 'Scenario 0 Tiers'] = 'EE + CH'
Tier.loc[(Tier['EES0'] != -1) & (Tier['EES0'] > 0) & (Tier['ESS0'] == 0) & (Tier['ECS0'] > 0) & (Tier['EFS0'] == 0),'Scenario 0 Tiers'] = 'EE Only'
Tier.loc[(Tier['EES0'] == 0), 'Scenario 0 Tiers'] = 'None'
Tier.loc[(Tier['EES0'] == -1), 'Scenario 0 Tiers'] = 'N/A'
# -- Scenario 1 -- #
Tier.loc[(Tier['EES1'] != -1) & (Tier['EES1'] > 0) & (Tier['ESS1'] > 0) & (Tier['ECS1'] > 0) & (Tier['EFS1'] > 0), 'Scenario 1 Tiers'] = '4 Tier' # All ON1
Tier.loc[(Tier['EES1'] != -1) & (Tier['EES1'] > 0) & (Tier['ESS1'] == 0) & (Tier['ECS1'] == 0) & (Tier['EFS1'] > 0), 'Scenario 1 Tiers'] = 'EE + Family'
Tier.loc[(Tier['EES1'] != -1) & (Tier['EES1'] > 0) & (Tier['ESS1'] > 0) & (Tier['ECS1'] == 0), 'Scenario 1 Tiers'] = 'EE + SP'
Tier.loc[(Tier['EES1'] != -1) & (Tier['EES1'] > 0) & (Tier['ESS1'] == 0) & (Tier['ECS1'] > 0), 'Scenario 1 Tiers'] = 'EE + CH'
Tier.loc[(Tier['EES1'] != -1) & (Tier['EES1'] > 0) & (Tier['ESS1'] == 0) & (Tier['ECS1'] > 0) & (Tier['EFS1'] == 0),'Scenario 1 Tiers'] = 'EE Only'
Tier.loc[(Tier['EES1'] == 0), 'Scenario 1 Tiers'] = 'None'
Tier.loc[(Tier['EES1'] == -1), 'Scenario 1 Tiers'] = 'N/A'

# -- Most Likely Scenario -- #
Tier.loc[(Tier['EEML'] != -1) & (Tier['EEML'] > 0) & (Tier['ESML'] > 0) & (Tier['ECML'] > 0) & (Tier['EFML'] > 0), 'Most Likely Tiers'] = '4 Tier' # All ON
Tier.loc[(Tier['EEML'] != -1) & (Tier['EEML'] > 0) & (Tier['ESML'] == 0) & (Tier['ECML'] == 0) & (Tier['EFML'] > 0), 'Most Likely Tiers'] = 'EE + Family'
Tier.loc[(Tier['EEML'] != -1) & (Tier['EEML'] > 0) & (Tier['ESML'] > 0) & (Tier['ECML'] == 0), 'Most Likely Tiers'] = 'EE + SP'
Tier.loc[(Tier['EEML'] != -1) & (Tier['EEML'] > 0) & (Tier['ESML'] == 0) & (Tier['ECML'] > 0), 'Most Likely Tiers'] = 'EE + CH'
Tier.loc[(Tier['EEML'] != -1) & (Tier['EEML'] > 0) & (Tier['ESML'] == 0) & (Tier['ECML'] > 0) & (Tier['EFML'] == 0),'Most Likely Tiers'] = 'EE Only'
Tier.loc[(Tier['EEML'] == 0), 'Most Likely Tiers'] = 'None'
Tier.loc[(Tier['EEML'] == -1), 'Most Likely Tiers'] = 'N/A'


# -- Sold Scenario -- #
Tier.loc[(Tier['EES'] != -1) & (Tier['EES'] > 0) & (Tier['ESS'] > 0) & (Tier['ECS'] > 0) & (Tier['EFS'] > 0), 'SOLD Tiers'] = '4 Tier' # All ON
Tier.loc[(Tier['EES'] != -1) & (Tier['EES'] > 0) & (Tier['ESS'] == 0) & (Tier['ECS'] == 0) & (Tier['EFS'] > 0), 'SOLD Tiers'] = 'EE + Family'
Tier.loc[(Tier['EES'] != -1) & (Tier['EES'] > 0) & (Tier['ESS'] > 0) & (Tier['ECS'] == 0), 'SOLD Tiers'] = 'EE + SP'
Tier.loc[(Tier['EES'] != -1) & (Tier['EES'] > 0) & (Tier['ESS'] == 0) & (Tier['ECS'] > 0), 'SOLD Tiers'] = 'EE + CH'
Tier.loc[(Tier['EES'] != -1) & (Tier['EES'] > 0) & (Tier['ESS'] == 0) & (Tier['ECS'] > 0) & (Tier['EFS'] == 0),'SOLD Tiers'] = 'EE Only'
Tier.loc[(Tier['EES'] == 0), 'SOLD Tiers'] = 'None'
Tier.loc[(Tier['EES'] == -1), 'SOLD Tiers'] = 'N/A'

# -- Tests -- #
Tier.loc[(Tier['Scenario 0 Tiers'] == 'N/A'), 'Result'] = 'Pass'
Tier.loc[(Tier['Scenario 0 Tiers'] == Tier['Scenario 1 Tiers']) & (Tier['Scenario 0 Tiers'] == Tier['Most Likely Tiers']) & (Tier['Scenario 0 Tiers'] == Tier['SOLD Tiers']), 'Result'] = 'Pass'
Tier.loc[(Tier['Scenario 1 Tiers'] == 'N/A') & (Tier['Scenario 0 Tiers'] == Tier['Most Likely Tiers']) & (Tier['Scenario 0 Tiers'] == Tier['SOLD Tiers']), 'Result'] = 'Pass'
Tier.loc[(Tier['Scenario 1 Tiers'] == 'N/A') & (Tier['SOLD Tiers'] == 'N/A') & (Tier['Scenario 0 Tiers'] == Tier['Most Likely Tiers']) , 'Result'] = 'Pass'
Tier.loc[(Tier['Most Likely Tiers'] == 'N/A') & (Tier['SOLD Tiers'] == 'N/A') & (Tier['Scenario 0 Tiers'] == Tier['Scenario 1 Tiers']) , 'Result'] = 'Pass'
Tier.loc[(Tier['Most Likely Tiers'] == 'N/A') & (Tier['SOLD Tiers'] == 'N/A') & (Tier['Scenario 1 Tiers'] == 'N/A') , 'Result'] = 'Pass'
Tier.loc[(Tier['SOLD Tiers'] == 'N/A') & (Tier['Scenario 0 Tiers'] == Tier['Scenario 1 Tiers']) & (Tier['Scenario 0 Tiers'] == Tier['Most Likely Tiers']) , 'Result'] = 'Pass'
Tier.loc[(Tier['Result'].isnull()) , 'Result'] = 'Fail'


#Tier[:] = np.where(Tier == -1,'', Tier)
#Tier= Tier.replace('-1','')

#Tier['EES0']=Tier['EES0'].replace('-1','')
#Tier['EES1']=Tier['EES1'].str.replace('-1','')
#Tier['EEML']=Tier['EEML'].str.replace('-1','')
#Tier['EES']=Tier['EES'].str.replace('-1','')
#Tier['ESS0']=Tier['ESS0'].str.replace('-1','')
#Tier['ESS1']=Tier['ESS1'].str.replace('-1','')
#Tier['ESML']=Tier['ESML'].str.replace('-1','')
#Tier['ESS']=Tier['ESS'].str.replace('-1','')
#Tier['ECS0']=Tier['ECS0'].str.replace('-1','')
#Tier['ECS1']=Tier['ECS1'].str.replace('-1','')
#Tier['ECML']=Tier['ECML'].str.replace('-1','')
#Tier['ECS']=Tier['ECS'].str.replace('-1','')
#Tier['EFS0']=Tier['EFS0'].str.replace('-1','')
#Tier['EFS1']=Tier['EFS1'].str.replace('-1','')
#Tier['EFML']=Tier['EFML'].str.replace('-1','')
#Tier['EFS']=Tier['EFS'].str.replace('-1','')


## -- Test Result -- ##
Tier['FailReason']='Tiered census does not match' 
Tier['Scenario']='N/A' 
df12_con=Tier

Tier.to_csv('TieredCensusMatch12.csv')
with open('TieredCensusMatch12.csv', 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write('Test #12: Check that tiered census is consistent for Scenario 0, Scenario 1, Most Likely, and Sold' + '\n' +'\n' + content)

# ESL Data Check #13

In [26]:
df13_con = df1[['AccountID','AccountName','Description','QuoteStatusDescription','Scenario',
                'OptionSequence','Status','EffectiveDate','UnderwriterName','UwrAssistantName',
                'RenewalTier','TreatyDescription']].drop_duplicates()

df13_con = df13_con[(df13_con['OptionSequence']==1) & (df13_con['RenewalTier'] == 'UNKN')
                 & ((df13_con['QuoteStatusDescription'].astype(str).str.startswith('Declined') == True) | 
                    (df13_con['QuoteStatusDescription'].astype(str).str.startswith('Pended') == True) | 
                    (df13_con['QuoteStatusDescription'].astype(str).str.startswith('Quoted') == True) | 
                    (df13_con['QuoteStatusDescription'].astype(str).str.startswith('Rescinded') == True) | 
                    (df13_con['QuoteStatusDescription'].astype(str).str.startswith('Sold') == True))]

df13_con['FailReason']='Does not have a treaty selected'
df13_con.to_csv('Draft13.csv')
#with open('StrategicPartner1.csv', 'r+') as f:
#    content = f.read()
#    f.seek(0, 0)
#    f.write('Test #1: Groups Missing a Strategic Partner' + '\n' +'\n' + content)
#df1_con

KeyError: "['TreatyDescription'] not in index"

In [ ]:
CombinedList = df1_con.append([df2_con,df3_con,df4_con,df5_con,df6_con,df8_con,df9_con,df10_con,df11_con,df12_con])
CombinedList = CombinedList[['AccountName','Description','Scenario', 'OptionSequence', 'Status', 'EffectiveDate', 'UnderwriterName', 'UwrAssistantName', 'QuoteStatusDescription','FailReason']].drop_duplicates()
CombinedList.reset_index(drop=True, inplace = True)
CombinedList.to_csv('CombinedList.csv')
InUW = CombinedList[(CombinedList['QuoteStatusDescription']=='Declined')]
InUW = InUW[['UwrAssistantName','AccountName','Description','Scenario','Status','EffectiveDate', 'UwrAssistantName', 'QuoteStatusDescription','FailReason']].drop_duplicates()
InUW.to_csv('InUw.csv', index= False)
InUW =pd.read_csv('InUw.csv')

Other = CombinedList
Other = Other[(Other['QuoteStatusDescription'] != 'Declined')]
Other = Other[['AccountName','Scenario', 'OptionSequence', 'Status', 'EffectiveDate', 'UnderwriterName','UwrAssistantName', 'QuoteStatusDescription','FailReason']]
Other.to_csv('Other.csv')
Other
InUWp

In [ ]:
InUW

### Merge the results together - Still need to add new tests

In [ ]:
df_total = df1[['AccountName','Status','EffectiveDate']].drop_duplicates()

In [ ]:
df_total.shape

In [ ]:
df_total0 = df_total.merge(df1_con,on=['AccountName','Status','EffectiveDate'],how='left',indicator=True)

In [ ]:
df_total0['_merge'].value_counts()

In [ ]:
df_total0['test 1 result'] = ''
df_total0.loc[df_total0['_merge'] == 'left_only','test 1 result'] = 'Pass'
df_total0.loc[df_total0['_merge'] == 'both','test 1 result'] = 'Fail'

In [ ]:
df_total0 = df_total0.drop(columns = '_merge')

In [ ]:
df_total1 = df_total0.merge(df2_con,on=['AccountName','Scenario','OptionSequence','Status','EffectiveDate'],how='left',indicator=True)

In [ ]:
df_total1['_merge'].value_counts()

In [ ]:
df_total1['test 2 result'] = ''

In [ ]:
df_total1.loc[df_total1['_merge'] == 'left_only','test 2 result'] = 'Pass'

In [ ]:
df_total1.loc[df_total1['_merge'] == 'both','test 2 result'] = 'Fail'

In [ ]:
df_total1 = df_total1.drop(columns = '_merge')

In [ ]:
df_total2 = df_total1.merge(df3_con,on=['AccountName','Scenario','OptionSequence','Status','EffectiveDate'],how='left',indicator=True)

In [ ]:
df_total2['_merge'].value_counts()

In [ ]:
df_total2['test 3 result'] = ''

In [ ]:
df_total2.loc[df_total2['_merge'] == 'left_only','test 3 result'] = 'Pass'

In [ ]:
df_total2.loc[df_total2['_merge'] == 'both','test 3 result'] = 'Fail'

In [ ]:
df_total2 = df_total2.drop(columns = ['_merge'])

In [ ]:
df_final = df_total2[(df_total2['test 1 result'] =='Fail') | (df_total2['test 2 result'] == 'Fail') | (df_total2['test 3 result'] == 'Fail')]

In [ ]:
df_final.shape

In [ ]:
df_final.to_excel('data_check_result_for_test1&2&3_20220113.xlsx',index=False)

In [ ]:
#Count of groups Missing Strategic Partner - Edited to show only Option 1

df1_con.loc[(df1_con['QuoteStatusDescription'] == 'In Underwriting'), 'Mailto'] = df1_con['UwrAssistantName']
df1_con.loc[(df1_con['QuoteStatusDescription'] != 'In Underwriting'), 'CCto'] = df1_con['UwrAssistantName']
df1_con.loc[(df1_con['QuoteStatusDescription'] != 'In Underwriting'), 'Mailto'] = df1_con['UnderwriterName']
df1_con['Subject']= 'AUTOMATED MESSAGE: '+ df1_con['AccountName'] + ' was set up without a Broker Compensation selection'
df1_con['Body']= 'Hello,<br><br>'+ df1_con['AccountName'] + ' was set up without a Broker Compensation selection. This field always needs'\
                 ' to have something selected. Please select a valid broker compensation agreement.<br><br> '\
                 '* If the broker does not have special comp "Null" should be selected.<br>'\
                 '* If group has not been quoted, please correct all open scenarios.<br>' \
                 '* If group is New Biz and has been quoted, please copy the quoted scenario and fix in the open scenario. If you used STS or rate to EAM to calculate rates, your final rates may have been changed and will need to be set back to the rates quoted previously. <br>' \
                 '* If case is a renewal.  Please correct all scenarios. <br>' \
                 '* If case has been quoted, please copy the most  recent issued scenario, correct and reissue at the same rates as before.  '
